## Data Prep 

## import libraries

In [35]:
import pandas as pd
import numpy as np

import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from imblearn.under_sampling import RandomUnderSampler

### Load data

In [2]:

df00 = pd.read_csv('input/df.csv')

df00.head()

,id,loan_amnt,term,emp_length,home_ownership,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,target
0,68407277,3600.0,1.0,10.0,MORTGAGE,55000.0,45,5.91,4.0,1506.0,4.0,0
1,68355089,24700.0,1.0,10.0,MORTGAGE,65000.0,6,16.06,0.0,57830.0,20.0,0
2,68341763,20000.0,2.0,10.0,MORTGAGE,63000.0,47,10.78,10.0,2737.0,4.0,0
3,68476807,10400.0,2.0,3.0,MORTGAGE,104433.0,45,25.37,1.0,4567.0,7.0,0
4,68426831,11950.0,1.0,4.0,RENT,34000.0,43,10.20,NaN,844.0,4.0,0


### Load metadata

In [3]:
metadata = pd.read_csv('artifacts/metadata.csv')
metadata

,Features,Role,Level,Keep,Tipo,Cardinality
0,id,id,nominal,False,object,1345310
1,loan_amnt,input,interval,True,float64,1556
2,term,input,interval,True,float64,2
3,emp_length,input,interval,True,float64,11
4,home_ownership,input,nominal,True,object,6
5,annual_inc,input,interval,True,float64,64362
6,addr_state,input,ordinal,True,int64,51
7,dti,input,interval,True,float64,7067
8,mths_since_recent_inq,input,interval,True,float64,26
9,bc_open_to_buy,input,interval,True,float64,74924


## Numerical Data

### Selecting features (Ordinal+Interval)

In [4]:
# Filtering Data Frame 
vars_numericas_df = metadata[((metadata.Level  == 'ordinal')|(metadata.Level == 'interval')) & (metadata.Role == 'input')]
vars_numericas_df

,Features,Role,Level,Keep,Tipo,Cardinality
1,loan_amnt,input,interval,True,float64,1556
2,term,input,interval,True,float64,2
3,emp_length,input,interval,True,float64,11
5,annual_inc,input,interval,True,float64,64362
6,addr_state,input,ordinal,True,int64,51
7,dti,input,interval,True,float64,7067
8,mths_since_recent_inq,input,interval,True,float64,26
9,bc_open_to_buy,input,interval,True,float64,74924
10,num_op_rev_tl,input,interval,True,float64,74


In [5]:
# list of selecting features
lista_vars_numericas = list(vars_numericas_df['Features'])
lista_vars_numericas

['loan_amnt',
 'term',
 'emp_length',
 'annual_inc',
 'addr_state',
 'dti',
 'mths_since_recent_inq',
 'bc_open_to_buy',
 'num_op_rev_tl']

In [6]:
df01 = df00[lista_vars_numericas]
df01.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl
0,3600.0,1.0,10.0,55000.0,45,5.91,4.0,1506.0,4.0
1,24700.0,1.0,10.0,65000.0,6,16.06,0.0,57830.0,20.0
2,20000.0,2.0,10.0,63000.0,47,10.78,10.0,2737.0,4.0
3,10400.0,2.0,3.0,104433.0,45,25.37,1.0,4567.0,7.0
4,11950.0,1.0,4.0,34000.0,43,10.20,NaN,844.0,4.0


### 1)  Treating Missing Values

In [7]:
# Verifying amount missing values
df01_missing = df01.isnull().sum()
df01_missing

loan_amnt                     0
term                          0
emp_length                78511
annual_inc                    0
addr_state                    0
dti                         374
mths_since_recent_inq    174071
bc_open_to_buy            61143
num_op_rev_tl             67527
dtype: int64

In [8]:
# count rows of dataframe
qt_rows = df00.shape[0]

# dataframe with quantity of missing values by feature
df_pct_missing = pd.DataFrame(df01_missing,columns=['qt_missing'])
df_pct_missing = pd.DataFrame(df01_missing,columns=['qt_missing'])
df_pct_missing['Features'] = df_pct_missing.index
df_pct_missing['pc_miss'] = (100*df_pct_missing['qt_missing'].divide(qt_rows)).astype(int)
df_pct_missing['qt_rows'] = qt_rows
df_pct_missing.reset_index(drop = True, inplace = True)

df_pct_missing

,qt_missing,Features,pc_miss,qt_rows
0,0,loan_amnt,0,1345310
1,0,term,0,1345310
2,78511,emp_length,5,1345310
3,0,annual_inc,0,1345310
4,0,addr_state,0,1345310
5,374,dti,0,1345310
6,174071,mths_since_recent_inq,12,1345310
7,61143,bc_open_to_buy,4,1345310
8,67527,num_op_rev_tl,5,1345310


### Save mean to fill NaN values during deployment.

In [9]:

mean_vars_num = df01[lista_vars_numericas].mean()

joblib.dump(mean_vars_num, "artifacts/mean_vars_num.joblib")


['artifacts/mean_vars_num.joblib']

In [10]:
# Replace missing values by mean 
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html

df02 = (df01.fillna(df01[lista_vars_numericas].mean())).astype(float)


In [11]:
print('Verify missing after treatment:')
df02.isnull().sum()

Verify missing after treatment:


loan_amnt                0
term                     0
emp_length               0
annual_inc               0
addr_state               0
dti                      0
mths_since_recent_inq    0
bc_open_to_buy           0
num_op_rev_tl            0
dtype: int64

In [12]:
df02.head(10)

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl
0,3600.0,1.0,10.0,55000.0,45.0,5.91,4.000000,1506.0,4.0
1,24700.0,1.0,10.0,65000.0,6.0,16.06,0.000000,57830.0,20.0
2,20000.0,2.0,10.0,63000.0,47.0,10.78,10.000000,2737.0,4.0
3,10400.0,2.0,3.0,104433.0,45.0,25.37,1.000000,4567.0,7.0
4,11950.0,1.0,4.0,34000.0,43.0,10.20,6.713209,844.0,4.0
5,20000.0,1.0,10.0,180000.0,34.0,14.67,10.000000,0.0,9.0
6,20000.0,1.0,10.0,85000.0,25.0,17.61,8.000000,13674.0,3.0
7,10000.0,1.0,6.0,85000.0,45.0,13.07,1.000000,8182.0,13.0
8,8000.0,1.0,10.0,42000.0,14.0,34.80,10.000000,9966.0,5.0
9,1400.0,1.0,3.0,64000.0,41.0,34.95,18.000000,7940.0,12.0


# Categorical Features (String/Char) 

### 1) Selecting nominal features

In [13]:
# Filter Data Frame 
vars_char_df = metadata[(metadata.Level  == 'nominal') & (metadata.Role == 'input')]
vars_char_df

,Features,Role,Level,Keep,Tipo,Cardinality
4,home_ownership,input,nominal,True,object,6


In [14]:
# list of cat features
lista_vars_char = list(vars_char_df['Features'])
lista_vars_char

['home_ownership']

### Create a dataframe with only categorical features

In [15]:
df06 = df00[lista_vars_char]
df06.head()

,home_ownership
0,MORTGAGE
1,MORTGAGE
2,MORTGAGE
3,MORTGAGE
4,RENT


### 2) Verify missing values

In [16]:
df06_missing = df06.isnull().sum()
df06_missing

home_ownership    0
dtype: int64

### 3) Treat missing values (replace by MISS category)


In [17]:
df07 = df06.fillna('MISS')
df07.head()

,home_ownership
0,MORTGAGE
1,MORTGAGE
2,MORTGAGE
3,MORTGAGE
4,RENT


In [18]:
df07_missing = df07.isnull().sum()
df07_missing

home_ownership    0
dtype: int64

In [19]:
df07.home_ownership.value_counts(dropna=False)

home_ownership
MORTGAGE    665579
RENT        534421
OWN         144832
ANY            286
OTHER          144
NONE            48
Name: count, dtype: int64

In [20]:
df07.home_ownership.replace({"ANY":"OTHERS","OTHER":"OTHERS","NONE":"OTHERS"},inplace=True)
df07.home_ownership.value_counts()

home_ownership
MORTGAGE    665579
RENT        534421
OWN         144832
OTHERS         478
Name: count, dtype: int64

### 3) Create Dummies features
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [21]:
# Filter Data Frame 
vars_to_dummif_df = metadata[(metadata.Level  == 'nominal') & (metadata.Role == 'input')& (metadata.Cardinality <= 20)]
vars_to_dummif_df

,Features,Role,Level,Keep,Tipo,Cardinality
4,home_ownership,input,nominal,True,object,6


In [22]:
# list of cat features
lista_vars_dummif = list(vars_to_dummif_df['Features'])
lista_vars_dummif

['home_ownership']

In [23]:
df08 = df07[lista_vars_dummif]
df08.head()

,home_ownership
0,MORTGAGE
1,MORTGAGE
2,MORTGAGE
3,MORTGAGE
4,RENT


In [24]:
# Generate dummies
df09 = pd.get_dummies(df08, 
                      columns=lista_vars_dummif,
                      drop_first=False, 
                      prefix = lista_vars_dummif,
                      prefix_sep='_',
                      dtype=int)
df09.head()

,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,0,0,0,1


### 4) Join all treat dataframe 

* df02 --> non standalization

* df09 --> dummies


In [26]:
df3_9 = pd.merge(df02,df09, left_index=True, right_index=True)
df_expl_not_norm = df3_9.copy()
df_expl_not_norm.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT
0,3600.0,1.0,10.0,55000.0,45.0,5.91,4.000000,1506.0,4.0,1,0,0,0
1,24700.0,1.0,10.0,65000.0,6.0,16.06,0.000000,57830.0,20.0,1,0,0,0
2,20000.0,2.0,10.0,63000.0,47.0,10.78,10.000000,2737.0,4.0,1,0,0,0
3,10400.0,2.0,3.0,104433.0,45.0,25.37,1.000000,4567.0,7.0,1,0,0,0
4,11950.0,1.0,4.0,34000.0,43.0,10.20,6.713209,844.0,4.0,0,0,0,1


In [28]:
df_not_norm = pd.merge(df_expl_not_norm,df00[['id','target']], left_index=True, right_index=True)
df_not_norm.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,id,target
0,3600.0,1.0,10.0,55000.0,45.0,5.91,4.000000,1506.0,4.0,1,0,0,0,68407277,0
1,24700.0,1.0,10.0,65000.0,6.0,16.06,0.000000,57830.0,20.0,1,0,0,0,68355089,0
2,20000.0,2.0,10.0,63000.0,47.0,10.78,10.000000,2737.0,4.0,1,0,0,0,68341763,0
3,10400.0,2.0,3.0,104433.0,45.0,25.37,1.000000,4567.0,7.0,1,0,0,0,68476807,0
4,11950.0,1.0,4.0,34000.0,43.0,10.20,6.713209,844.0,4.0,0,0,0,1,68426831,0


## Split data into training, validation and test sets, ensuring the class distribution is maintained


In [29]:
df_not_norm.set_index('id',inplace=True)


In [30]:
df_not_norm.to_csv('input/df_not_norm.csv', sep=',', encoding='utf-8',index=True)


In [32]:
# Drop the 'user_id' and 'IsBuyer' column to prepare data for splitting
data = df_not_norm.drop(columns=['target'])

# Get the target variable
answer = df_not_norm['target']



# Split data into training, validation and test sets

X_trainval, X_test, y_trainval, y_test = train_test_split(data, answer
                                                          , test_size=0.2
                                                          , stratify=df_not_norm['target']
                                                          , random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval
                                                  , test_size=0.25
                                                  , stratify=y_trainval
                                                  , random_state=42)



In [33]:
X_train[lista_vars_numericas].head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl
id,,,,,,,,,
105253408,16000.0,1.0,7.000000,84996.0,43.0,22.91,21.0,996.0,3.0
16592425,19200.0,2.0,5.965873,62960.0,30.0,19.06,1.0,19930.0,17.0
58470345,6000.0,1.0,7.000000,30000.0,51.0,6.48,3.0,0.0,3.0
10092824,10000.0,1.0,7.000000,150000.0,34.0,10.81,11.0,122590.0,34.0
65965690,21000.0,1.0,10.000000,144000.0,18.0,20.84,6.0,18590.0,7.0


In [37]:
# Initialize the StandardScaler object and fit it to the training data


scaler = StandardScaler()
scaler.fit(X_train[lista_vars_numericas])

# After fitting the scaler to your data as you did:
joblib.dump(scaler,"artifacts/scaler.joblib")

X_train_std = X_train.copy()
X_val_std = X_val.copy()
X_test_std = X_test.copy()

# Scale the training, validation, and test sets using the scaler
X_train_std[lista_vars_numericas] = scaler.transform(X_train[lista_vars_numericas])
X_val_std[lista_vars_numericas] = scaler.transform(X_val[lista_vars_numericas])
X_test_std[lista_vars_numericas] = scaler.transform(X_test[lista_vars_numericas])

### 5) Bring the target to create Train, Val e Test dataset 

In [38]:
Train = X_train_std.copy()
Train['target'] = y_train

Val = X_val_std.copy()
Val['target'] = y_val

Test = X_test_std.copy()
Test['target'] = y_test

Train.shape, Val.shape, Test.shape


((807186, 14), (269062, 14), (269062, 14))

## Verify buyer proportion

In [40]:
from collections import Counter
print(f'Target proportion for Original dataset is {Counter(answer)} and BadRate is {round(100*(Counter(answer)[1.0]/(Counter(answer)[0.0]+Counter(answer)[1.0])),2)}%')
print(f'Target proportion for Train dataset is {Counter(y_train)} and BadRate is {round(100*(Counter(y_train)[1.0]/(Counter(y_train)[0.0]+Counter(y_train)[1.0])),2)}%')
print(f'Target proportion for Validation dataset is {Counter(y_val)} and BadRate is {round(100*(Counter(y_val)[1.0]/(Counter(y_val)[0.0]+Counter(y_val)[1.0])),2)}%')
print(f'Target proportion for Test dataset is {Counter(y_test)} and BadRate is {round(100*(Counter(y_test)[1.0]/(Counter(y_test)[0.0]+Counter(y_test)[1.0])),2)}%')

Target proportion for Original dataset is Counter({0: 1076751, 1: 268559}) and BadRate is 19.96%
Target proportion for Train dataset is Counter({0: 646051, 1: 161135}) and BadRate is 19.96%
Target proportion for Validation dataset is Counter({0: 215350, 1: 53712}) and BadRate is 19.96%
Target proportion for Test dataset is Counter({0: 215350, 1: 53712}) and BadRate is 19.96%


In [41]:
Train.head()

,loan_amnt,term,emp_length,annual_inc,addr_state,dti,mths_since_recent_inq,bc_open_to_buy,num_op_rev_tl,home_ownership_MORTGAGE,home_ownership_OTHERS,home_ownership_OWN,home_ownership_RENT,target
id,,,,,,,,,,,,,,
105253408,0.179897,-0.564595,0.289061,0.119112,0.231563,0.410550,2.617142,-0.612313,-1.188055,1,0,0,0,1
16592425,0.546808,1.771181,0.000214,-0.183949,-0.964507,0.069240,-1.046230,0.647751,1.963171,0,0,1,0,0
58470345,-0.966701,-0.564595,0.289061,-0.637249,0.967607,-1.046003,-0.679893,-0.678597,-1.188055,0,0,0,1,0
10092824,-0.508062,-0.564595,0.289061,1.013113,-0.596485,-0.662140,0.785456,7.479804,5.789659,1,0,0,0,0
65965690,0.753196,-0.564595,1.127005,0.930595,-2.068572,0.227040,-0.130387,0.558573,-0.287704,1,0,0,0,1


### 6) Save final data


In [109]:
# Vamos utilizar o formato csv porque esta tabela é pequena em linhas e colunas 
Train.to_csv('input/Train.csv', sep=',', encoding='utf-8',index=True)
Val.to_csv('input/Val.csv', sep=',', encoding='utf-8',index=True)
Test.to_csv('input/Test.csv', sep=',', encoding='utf-8',index=True)